# Command Pattern

In [1]:
from abc import abstractmethod, ABCMeta

In [2]:
# 커맨드 추상 클래스
class Command(metaclass=ABCMeta):
    @abstractmethod
    def execute():
        pass

    @abstractmethod
    def undo():
        pass


In [3]:
# 조명
class Light:
    def __init__(self, light_name):
        self.light_name = light_name

    def on(self):
        print(f'{self.light_name} 조명을 켭니다.')
    
    def off(self):
        print(f'{self.light_name} 조명을 끕니다.')


# 오디오
class Stereo:
    def __init__(self, stereo_name):
        self.stereo_name = stereo_name
        self.volume = 10
    
    def on(self):
        print(f'{self.stereo_name}을 켭니다.')
    
    def off(self):
        print(f'{self.stereo_name}을 끕니다.')
    
    def set_cd(self):
        print(f'{self.stereo_name}가 CD를 재생합니다.')
    
    def set_radio(self):
        print(f'{self.stereo_name}가 라디오를 켭니다.')
    
    def set_volume(self, volume):
        self.volume = volume
        print(f'{self.stereo_name}의 볼륨을 {volume}으로 조절합니다.')
    

# 차고
class GarageDoor:
    def __init__(self, garage_door_name):
        self.garage_door_name = garage_door_name

    def open(self):
        print(f'{self.garage_door_name} 문을 엽니다.')
    
    def close(self):
        print(f'{self.garage_door_name} 문을 닫습니다.')


# 선풍기
class Fan:
    def __init__(self, fan_name):
        self.fan_name = fan_name
        self.speed = None
    
    def on(self):
        print(f'{self.fan_name}를 켭니다.')
    
    def off(self):
        print(f'{self.fan_name}를 끕니다.')
    
    def high(self):
        self.speed = 'high'
        print(f'{self.fan_name} 속도를 {self.speed}로 설정합니다.')
    
    def medium(self):
        self.speed = 'medium'
        print(f'{self.fan_name} 속도를 {self.speed}로 설정합니다.')
    
    def low(self):
        self.speed = 'low'
        print(f'{self.fan_name} 속도를 {self.speed}로 설정합니다.')
    
    def get_speed(self):
        return self.speed


In [8]:
# 조명을 켜는 커맨드
class LightOnCammand(Command):
    def __init__(self, light):
        '''
        light의 종류를 저장합니다.
        '''
        self.light = light

    def execute(self):
        self.light.on()
    
    def undo(self):
        self.light.off()


# 조명을 끄는 커맨드
class LightOffCammand(Command):
    def __init__(self, light):
        '''
        light의 종류를 저장합니다.
        '''
        self.light = light

    def execute(self):
        self.light.off()
    
    def undo(self):
        self.light.on()


# 차고 문을 여는 커맨드
class GarageDoorOpenCommand(Command):
    def __init__(self, garage):
        '''
        Garage의 종류를 저장합니다.
        '''
        self.garage = garage

    def execute(self):
        self.garage.open()
    
    def undo(self):
        self.garage.close()


# 차고 문을 닫는 커맨드
class GarageDoorCloseCommand(Command):
    def __init__(self, garage):
        '''
        Garage의 종류를 저장합니다.
        '''
        self.garage = garage

    def execute(self):
        self.garage.close()
    
    def undo(self):
        self.garage.open()


# 오디오를 켜는 커맨드(CD 자동 재생)
class StereoOnWithCDCommand(Command):
    def __init__(self, stereo):
        '''
        오디오 종류를 저장합니다.
        '''
        self.stereo = stereo

    def execute(self):
        self.stereo.on()
        self.stereo.set_cd()
        self.stereo.set_volume(11)
    
    def undo(self):
        self.stereo.off()


# 오디오를 끄는 커맨드
class StereoOffCommand(Command):
    def __init__(self, stereo):
        '''
        오디오 종류를 저장합니다.
        '''
        self.stereo = stereo

    def execute(self):
        self.stereo.off()
    
    def undo(self):
        self.stereo.on()
        self.stereo.set_cd()
        self.stereo.set_volume(11)


# 선풍기를 high로 켜는(바꾸는) 커맨드
class FanHighCommand(Command):
    def __init__(self, fan):
        '''
        선풍기 종류를 저장합니다.
        '''
        self.fan = fan
        self.speed = None

    def execute(self):
        self.prev_speed = self.fan.get_speed()
        self.fan.high()
    
    def undo(self):
        if self.prev_speed is None:
            self.fan.off()
        elif self.prev_speed == 'medium':
            self.fan.mudium()
        elif self.prev_speed == 'low':
            self.fan.low()


# 선풍기를 끄는 메서드
class FanOffCommand(Command):
    def __init__(self, fan):
        '''
        선풍기 종류를 저장합니다.
        '''
        self.fan = fan
        self.speed = None
    
    def execute(self):
        self.prev_speed = self.fan.get_speed()
        self.fan.off()
    
    def undo(self):
        if self.prev_speed == 'high':
            self.fan.high()
        elif self.prev_speed == 'medium':
            self.fan.mudium()
        elif self.prev_speed == 'low':
            self.fan.low()


In [11]:
# 리모컨
class RemoteControl:
    def __init__(self):
        # on command 리스트 초기화
        self.on_commands = [None] * 4

        # off command 리스트 초기화
        self.off_commands = [None] * 4

        # 마지막으로 사용한 커맨드
        self.undo_command = None
    
    def set_command(self, slot, on_command, off_command):
        '''
        커맨드를 설정합니다.
        '''

        self.on_commands[slot] = on_command
        self.off_commands[slot] = off_command
    
    def on_button_was_pushed(self, slot):
        self.on_commands[slot].execute()
        self.undo_command = self.on_commands[slot]
    
    def off_button_was_pushed(self, slot):
        self.off_commands[slot].execute()
        self.undo_command = self.off_commands[slot]
    
    def undo_button_was_pushed(self):
        self.undo_command.undo()
    
    def __str__(self) -> str:
        message = '------------------------------리모컨------------------------------\n'
        
        for slot in range(len(self.on_commands)):
            message += f'slot {slot}: {self.on_commands[slot].__class__.__name__}'.ljust(40) + f'{self.off_commands[slot].__class__.__name__}\n'
        
        return message

In [12]:
living_room_light = Light('living_room')
front_garage_door = GarageDoor('front_garage_door')
car_stereo = Stereo('car_stereo')
ceiling_fan = Fan('ceiling_fan')

living_room_light_on = LightOnCammand(living_room_light)
living_room_light_off = LightOffCammand(living_room_light)
front_garage_door_open = GarageDoorOpenCommand(front_garage_door)
front_garage_door_close = GarageDoorCloseCommand(front_garage_door)
car_stereo_on_with_cd = StereoOnWithCDCommand(car_stereo)
car_stereo_off = StereoOffCommand(car_stereo)
ceiling_fan_high = FanHighCommand(ceiling_fan)
ceiling_fan_off = FanOffCommand(ceiling_fan)

remote_control = RemoteControl()
remote_control.set_command(0, living_room_light_on, living_room_light_off)
remote_control.set_command(1, front_garage_door_open, front_garage_door_close)
remote_control.set_command(2, car_stereo_on_with_cd, car_stereo_off)
remote_control.set_command(3, ceiling_fan_high, ceiling_fan_off)

print(remote_control)

------------------------------리모컨------------------------------
slot 0: LightOnCammand                  LightOffCammand
slot 1: GarageDoorOpenCommand           GarageDoorCloseCommand
slot 2: StereoOnWithCDCommand           StereoOffCommand
slot 3: FanHighCommand                  FanOffCommand



In [13]:
remote_control.on_button_was_pushed(0)
remote_control.off_button_was_pushed(0)
remote_control.undo_button_was_pushed()
remote_control.on_button_was_pushed(1)
remote_control.off_button_was_pushed(1)
remote_control.undo_button_was_pushed()
remote_control.on_button_was_pushed(2)
remote_control.off_button_was_pushed(2)
remote_control.undo_button_was_pushed()
remote_control.on_button_was_pushed(3)
remote_control.off_button_was_pushed(3)
remote_control.undo_button_was_pushed()

living_room 조명을 켭니다.
living_room 조명을 끕니다.
living_room 조명을 켭니다.
front_garage_door 문을 엽니다.
front_garage_door 문을 닫습니다.
front_garage_door 문을 엽니다.
car_stereo을 켭니다.
car_stereo가 CD를 재생합니다.
car_stereo의 볼륨을 11으로 조절합니다.
car_stereo을 끕니다.
car_stereo을 켭니다.
car_stereo가 CD를 재생합니다.
car_stereo의 볼륨을 11으로 조절합니다.
ceiling_fan 속도를 high로 설정합니다.
ceiling_fan를 끕니다.
ceiling_fan 속도를 high로 설정합니다.
